In [190]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

### Reading the text data

In [20]:
text_data_sentence = pd.read_csv('textdatanew.csv', encoding='ISO-8859-1')

In [25]:
text_data_sentence.head()

,CollectionID,BiographyID,ParagraphNo,ParagraphText
0,a001,bio04,1,"A FRENCH philosopher, moralizing on the great ..."
1,a001,bio04,2,Cleopatra was joint heir to the throne of Egyp...
2,a001,bio04,3,Cleopatra might have responded with a brillian...
3,a001,bio04,4,Caesar was then above fifty years of age. His ...
4,a001,bio04,5,For three years Cleopatra reigned with little ...


### Reading the Response file

In [39]:
bess_tags = pd.read_csv('CBW_Bess_tags_final2.csv')

In [42]:
bess_tags.head()

,Content,Event,Type,para no,URI,author,biographyID,collectionID,personaName,title
0,after,name,stageOfLife,1.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...
1,culmination,name,stageOfLife,1.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...
2,middle,name,stageOfLife,2.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...
3,middle,name,stageOfLife,3.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...
4,middle,name,stageOfLife,4.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...


In [48]:
#bess_tags.head()

In [56]:
bess_reponse = bess_tags.loc[:,['Content','Event','Type','para no','biographyID','collectionID']]

In [59]:
bess_reponse= bess_reponse.fillna(' ')

In [60]:
bess_reponse.loc[:,'Response'] = bess_reponse.loc[:,['Content','Event']].apply(lambda x: '_'.join(x),axis = 1)

In [85]:
top_10_events = bess_reponse[bess_reponse.Type.isin(['Event'])].Response.value_counts()[0:10]
top_10_events

city_locationSetting            1351
husband_agentType               1245
conversation_type               1118
nobleman, named_agentType        853
father_agentType                 818
lover, male, named_agentType     776
royalty, male_agentType          753
famous man_agentType             712
writing letter_type              667
sovereign, male_agentType        655
Name: Response, dtype: int64

In [72]:
bess_reponse.shape

(170663, 7)

In [ ]:
bess_reponse.loc[:,['para no','biographyID','collectionID','Response']].head()

In [81]:
bess_response_df = bess_reponse.loc[:,['para no','biographyID','collectionID','Response','Type']].drop_duplicates()

In [83]:
#bess_response_df.groupby(['para no','biographyID','collectionID','Type'],as_index = False)['Response'].apply(lambda x: ' '.join(x))

In [89]:
final_response_file = bess_response_df[bess_response_df.Response.isin(top_10_events.index)]

In [95]:
#final_response_file.melt()

In [101]:
final_response_file.head()

,para no,biographyID,collectionID,Response,Type,values
27,2,bio04,a001,"royalty, male_agentType",Event,1
29,2,bio04,a001,city_locationSetting,Event,1
35,3,bio04,a001,"royalty, male_agentType",Event,1
37,3,bio04,a001,city_locationSetting,Event,1
49,4,bio04,a001,"royalty, male_agentType",Event,1


In [98]:
final_response_file.loc[:,'values'] = 1

C:\Users\arvra\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [156]:
final_response_wide = (final_response_file.pivot_table(index=['para no','biographyID','collectionID','Type'],\
                                       columns='Response', values='values').reset_index())

In [157]:
final_response_wide = final_response_wide.fillna(0)

In [158]:
final_response_wide.head()

Response,para no,biographyID,collectionID,Type,city_locationSetting,conversation_type,famous man_agentType,father_agentType,husband_agentType,"lover, male, named_agentType","nobleman, named_agentType","royalty, male_agentType","sovereign, male_agentType",writing letter_type
0,1.0,bio01,a185,Event,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,bio01,a186,Event,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,bio01,a207,Event,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1.0,bio01,a221,Event,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,bio01,a270,Event,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [159]:
text_data_sentence_final = pd.merge(text_data_sentence,final_response_wide, how = 'left', left_on = ['CollectionID','BiographyID','ParagraphNo'],\
                             right_on = ['collectionID','biographyID','para no'])

In [160]:
text_data_sentence_final = text_data_sentence_final[~text_data_sentence_final['para no'].isna()]

In [161]:
text_data_sentence_final_response = text_data_sentence_final[text_data_sentence_final.columns[-10:]]

In [171]:
final_response_file.Response.value_counts()

city_locationSetting            1180
conversation_type               1055
husband_agentType               1034
father_agentType                 672
writing letter_type              640
royalty, male_agentType          638
nobleman, named_agentType        630
famous man_agentType             618
lover, male, named_agentType     583
sovereign, male_agentType        526
Name: Response, dtype: int64

#### Bag of Words

In [117]:
stop_words = stopwords.words('english')

In [118]:
bow_model = CountVectorizer(ngram_range= (1,2),stop_words=stop_words)

In [119]:
Para_text_bow = bow_model.fit_transform(text_data_sentence_final.ParagraphText)

#### Tf-IDF

In [121]:
tf_idf_model = TfidfVectorizer(ngram_range=(1,2),stop_words = stop_words)

In [122]:
Para_text_tfidf = tf_idf_model.fit_transform(text_data_sentence_final.ParagraphText)

### Splitting data into train and test

In [125]:
from sklearn.model_selection import train_test_split

In [197]:
X_train, X_test, y_train, y_test = train_test_split(Para_text_tfidf,text_data_sentence_final_response,
                                                    test_size = 0.3, random_state = 0)

In [198]:
y_test.shape

(1631, 10)

### Machine Learning Models

In [280]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [285]:
#knn_model = KNeighborsClassifier(n_neighbors= 3, p = 1.5)
rf_model = RandomForestClassifier()
lr_model = LogisticRegression(multi_class='multinomial')

In [286]:
#y_train.melt()[y_train.melt().value.isin([1])].variable.value_counts()

In [288]:
rf_model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [299]:
feature_importances = pd.DataFrame(rf_model.feature_importances_,
                                  index = bow_model.get_feature_names(),
                                  columns=['importance']).sort_values('importance',ascending=False)

feature_importances

,importance
king,0.004413
letter,0.003415
london,0.003034
wife,0.003014
thou,0.002648
years,0.002632
husband,0.002586
lord,0.002111
new,0.001882
cleopatra,0.001824


In [258]:
np.array(rf_model.predict_proba(X_test)).shape

(10, 1631, 2)

In [279]:
[np.argmax([int(each[1]>0.4) for each in np.array(rf_model.predict_proba(X_test))[:,each_example,:]]) if \
     sum([int(each[1]>0.4) for each in np.array(rf_model.predict_proba(X_test))[:,each_example,:]]) > 0 else 10
            for each_example in range(100)]

[1,
 1,
 1,
 10,
 1,
 10,
 1,
 1,
 10,
 1,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 0,
 10,
 10,
 0,
 10,
 10,
 10,
 1,
 1,
 10,
 0,
 4,
 10,
 10,
 0,
 1,
 0,
 0,
 10,
 10,
 10,
 10,
 10,
 10,
 1,
 10,
 10,
 1,
 0,
 10,
 10,
 4,
 10,
 0,
 10,
 10,
 1,
 10,
 10,
 10,
 9,
 10,
 1,
 10,
 10,
 10,
 10,
 6,
 10,
 10,
 5,
 10,
 0,
 10,
 1,
 10,
 10,
 1,
 0,
 4,
 10,
 0,
 10,
 1,
 10,
 0,
 10,
 10,
 9,
 1,
 4,
 6,
 10,
 10,
 10,
 10,
 10,
 1,
 10,
 1]

In [212]:
result = [np.argmax(each) if each.sum() != 0 else 10 for each in rf_model.predict_proba(X_test)]

In [215]:
pd.DataFrame(result)[0].value_counts()

10    1316
1      188
0       52
4       17
9       13
5       11
3       11
6        9
2        9
8        3
7        2
Name: 0, dtype: int64

In [205]:
y_test.melt()[y_test.melt().value.isin([1])].variable.value_counts()

city_locationSetting            328
conversation_type               302
husband_agentType               295
father_agentType                206
royalty, male_agentType         198
nobleman, named_agentType       196
writing letter_type             189
famous man_agentType            178
lover, male, named_agentType    168
sovereign, male_agentType       166
Name: variable, dtype: int64